In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

from chickenstats.chicken_nhl import Season, Scraper
from chickenstats.utilities import ChickenProgress

from pathlib import Path

import datetime as dt

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
predictions = pd.read_csv("./simulations/predictions_2025-01-18.csv")

team_strength_scores = pd.read_csv("./simulations/team_strength_scores_2025-01-18.csv")

In [ ]:
def process_predictions(
    predictions: pd.DataFrame, schedule: pd.DataFrame
) -> pd.DataFrame:
    """Docstring."""
    sim_games = schedule.loc[schedule.game_id.isin(predictions.game_id)].reset_index(
        drop=True
    )

    winners = np.where(
        sim_games.home_score > sim_games.away_score,
        sim_games.home_team,
        sim_games.away_team,
    )
    winners_dict = dict(zip(sim_games.game_id, winners))

    predictions["draw"] = np.where(predictions.home_win == predictions.away_win, 1, 0)

    group_list = ["game_id", "home_team", "away_team"]
    agg_stats_list = ["home_win", "away_win", "draw"]
    agg_stats_dict = {x: "sum" for x in agg_stats_list}

    pred_results = predictions.groupby(group_list, as_index=False).agg(agg_stats_dict)

    for stat in agg_stats_list:
        pred_results[f"{stat}_percent"] = pred_results[stat] / pred_results[
            agg_stats_list
        ].sum(axis=1)

    pred_results["predicted_winner"] = np.where(
        pred_results.home_win_percent > pred_results.away_win_percent,
        pred_results.home_team,
        pred_results.away_team,
    )
    pred_results["winner"] = pred_results.game_id.map(winners_dict)
    pred_results["prediction_correct"] = np.where(
        pred_results.predicted_winner == pred_results.winner, 1, 0
    )

    return pred_results

In [ ]:
days = [f"2025-01-{x}" for x in range(18, 21)]

concat_list = []

filepaths = [Path(f"./simulations/predictions_{x}.csv") for x in days]

for filepath in filepaths:
    df = pd.read_csv(filepath)
    concat_list.append(df)

pd.concat(concat_list)

In [ ]:
season = Season(2024)

In [ ]:
schedule = season.schedule()

In [ ]:
predictions = pd.concat(concat_list)

In [ ]:
pred_results = process_predictions(predictions=predictions, schedule=schedule)

In [ ]:
pred_results

In [ ]:
pred_results.prediction_correct.sum() / pred_results.shape[0]

In [ ]:
pred_results

In [ ]:
sim_games = schedule.loc[schedule.game_date == "2025-01-18"].reset_index(drop=True)

winners = np.where(
    sim_games.home_score > sim_games.away_score,
    sim_games.home_team,
    sim_games.away_team,
)

winners_dict = dict(zip(sim_games.game_id, winners))